## Bucky Brooks  
http://www.nfl.com/news/author?id=09000d5d80f97bfd  
Scrape all his articles above and look for draft grades

In [16]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib
from collections import defaultdict
import datetime
import string

In [2]:
writer_id="09000d5d80f97bfd"
base_url = 'http://www.nfl.com/news/author?id='+writer_id
html_week = urllib.urlopen(base_url).read()
soup = BeautifulSoup(html_week, 'html.parser')

In [3]:
articles=soup.select('div[class=news-author-blurb-container]')

In [9]:
def gradesArticles(writer_id):
    writer_id="09000d5d80f97bfd"
    base_url = 'http://www.nfl.com/news/author?id='+writer_id
    html_week = urllib.urlopen(base_url).read()
    soup = BeautifulSoup(html_week, 'html.parser')
    articles=soup.select('div[class=news-author-blurb-container]')
    gradeArticles=[]
    for article in articles:
        a=article.select("a")[0]
        title=a.text
        url=a.get("href")
        if 'draft grades' in title:
            gradeArticles.append({'url': url, 'title':title})
    return gradeArticles
grades=pd.DataFrame(gradesArticles(articles))

In [10]:
grades.tail()

,title,url
39,NFC North draft grades Plenty of picks poor pr...,/draft/story/09000d5d817615c8/article/nfc-nort...
40,AFC North draft grades Browns drop behind prud...,/draft/story/09000d5d81760c9e/article/afc-nort...
41,AFC South draft grades Colts have found hidden...,/draft/story/09000d5d81759716/article/afc-sout...
42,NFC East draft grades Redskins have little to ...,/draft/story/09000d5d81753c34/article/nfc-east...
43,AFC East draft grades Besides Gholston Jets ha...,/draft/story/09000d5d81753bf7/article/afc-east...


## Team Grades

In [12]:
def procLinks(paragraph):
    #can also return quote in entity graph form
    team=''
    players=[]
    for link in paragraph.select('a'):
        if 'team' in str(link):
            if link.text[0].isupper():
                team=link.text
        elif 'player' in str(link):
            players.append({'url': link.get('href'), 'name': link.text})
    return {'team': team, 'players': players}
    
    
def procBold(paragraph):
    grade=None
    for b in paragraph.select('b'):
        if 'GRADE: ' in b.text:
            grade=b.text.split('GRADE: ')[1]
    return {'grade': grade}
    
def getMeta(paragraph):
    linkData=procLinks(paragraph)
    boldData=procBold(paragraph)
    meta=dict(linkData.items()+boldData.items())
    return meta

In [13]:
rows=[]
for url in grades['url']:
    html = urllib.urlopen('http://www.nfl.com/'+url).read()
    soup=BeautifulSoup(html, 'html.parser')
    rawTime=soup.find(id='article-time').get('title')[0:10]
    date=datetime.datetime.strptime(rawTime, '%Y-%m-%d')
    for paragraph in soup.select('p'):
        if 'GRADE: ' in str(paragraph):
            row = getMeta(paragraph)
            row['published']=date
            rows.append(row)
bucky=pd.DataFrame(rows)

In [14]:
bucky=bucky.dropna(subset=['team', 'grade'])

In [36]:
gradeVals={'A': 4., 'B': 3., 'C': 2., 'D': 1., 'F': 0.}
def gradeToNum(grade):
    offset=0
    if grade[-1]=='+':
        offset=.33
    elif grade[-1]=='-':
        offset=-.33
    return offset+gradeVals[grade[0]]
bucky['gradeVal']=bucky['grade'].apply(gradeToNum)

In [85]:
bucky['year']=bucky['published'].dt.year

iterate through players and see if still playing or starting and then team record

In [98]:
bucky['gradeVals'].mean()

3.0551960784313721

In [129]:
time_series=[]
for idx, t in bucky.groupby('team', axis=0):
    sum_=0
    for year in [2015,2014,2013]:
        row=t[t['published'].dt.year==year]
        gradVal=0
        if len(row)==0:
            gradeVal=3.05
        else:
            gradeVal=row.iloc[0]['gradeVal']
        score=.8**(-2013+year)*gradeVal
        sum_+=score
    
    time_series.append((t['team'].iloc[0], sum_, len(t)))

In [130]:
teamGrades=pd.DataFrame(time_series)
teamGrades.sort(1, ascending=False)

C:\Users\Roger\Anaconda2\envs\py2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,0,1,2
35,Vikings,9.1252,4
26,Rams,8.4340,3
19,Jaguars,8.3348,3
29,SAN FRANCISCO 49ERS:,8.2020,1
5,Browns,8.1172,3
6,Buccaneers,8.1172,3
16,Falcons,8.0100,3
25,Raiders,7.9588,2
18,HOUSTON TEXANS:,7.9380,1
20,Jets,7.8940,4


Better teams are ranked worse. panthers had strong draft classes. Same with patriots. At least moreso than chargers and especially browns

for team grades need to compare this discounted score to records. Plot scatter plot of win percent and score. No coaching built in so perhaps poor correlation corresponds to coach skill or other factors

Ironically, http://www.nfl.com/news/story/0ap3000000664439/article/seahawks-steelers-panthers-headline-10-most-talented-teams